# DATA102 Homework 1: Web Scraping

**Group Number**: 3
**Members**:
- Jose Maria Angelo Guerra
- Kyle Carlo Lasala
- Katrina Bianca Roco
- Antonio Jose Maria Lorenzo
- Josh Angelo Theodore Borro
- Charles Joseph Hinolan

**Section**: S11

**Instructor**: Mr. Jude Michael Teves

### Import Libraries

In [133]:
import requests
import pandas as pd
import time

### Parse Using BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
page = "https://itch.io/games"
#contact is my personal email
headers = {"User-Agent": "EducationalScraper/1.0 (contact: hinolancj@gmail.com)"}

# Disallowed paths based on itch.io/robots.txt
disallowed_paths = ["/embed/", "/embed-upload/", "/search", "/checkout/", "/game/download/", "/bundle/download/", "/register-for-purchase/", "/email-feedback/"]

# Function to check if URL is allowed
def is_allowed(url):
    for path in disallowed_paths:
        if path in url:
            return False
    return True


url = f"{page}"
if is_allowed(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    print(soup.prettify())
    time.sleep(2)
else:
    print(f"Skipping disallowed URL: {url}")

### Setup Browser Automation

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.edge.service import Service #microsft edge, change to webdriver.chrome.service for chrome

driver_path = "C:/Users/Charles/Desktop/COLLEGE/DATA102/edgedriver_win64/msedgedriver.exe" #edit your driver's path
url = "https://itch.io/games" # edit me

service = Service(driver_path)
driver = webdriver.Edge(service=service)

driver.get(url)
print(driver.page_source)

### Extracting the Data

data to extract (for now):
1. game id (class=game_cell has_cover)
2. game title (class=game_title)
3. game text (class=game_text)
4. author (class=game_author)
5. platform (class=game_platform)
6. genre (class=game_genre)

-------------- need to click on the game to get the following data below--------------

7. rating
8. tags
9. average session (time)
10. inputs

In [136]:
#auto scrolling algorithm 
#TODO: make it retreive 1000 games
pause = 0.5
lastHeight = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(pause)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
        break
    lastHeight = newHeight

In [137]:
#retrieve games
games = driver.find_elements(By.XPATH, "//div[@class='game_cell has_cover']")

#create lists
id_list = []
title_list = []
genre_list = []
author_list = []
text_list = []

#extract game data
#NOTE: some games do not have all the data, so we need to account for that by adding N/A if the data is not present in order for the lists to be the same length
#TODO: probably a better way to do the conditions
for game in games:
    #all games are guaranteed to have a game_id
    game_id = game.get_attribute("data-game_id")
    id_list.append(game_id)

    title = game.find_elements(By.XPATH, ".//a[@class='title game_link']")
    genre = game.find_elements(By.XPATH, ".//div[@class='game_genre']")
    author = game.find_elements(By.XPATH, ".//div[@class='game_author']")
    text = game.find_elements(By.XPATH, ".//div[@class='game_text']")

    if title:
        title_list.append(title[0].text)
    else:
        title_list.append("N/A")

    if genre:
        genre_list.append(genre[0].text)
    else:
        genre_list.append("N/A")

    if author:
        author_list.append(author[0].text)
    else:
        author_list.append("N/A")

    if text:
        text_list.append(text[0].text)
    else:
        text_list.append("N/A")
    
print(id_list[0])    
print(title_list[0])
print(genre_list[0])
print(author_list[0]) 
print(text_list[0])


2955066
Incredibox - Sprunki
N/A
wolf_hal
N/A


In [138]:
print(id_list[1])
print(title_list[1])
print(genre_list[1])
print(author_list[1])
print(text_list[1])

1635465
"Voices Of The Void" Pre-Alpha
Simulation
mrdrnose
Gather unknown signals from deep, silent space


### Loading the Data into the DataFrame

### Exporting the DataFrame to CSV